In [1]:
library(tidyverse)
library(MASS)
library(Metrics)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.2     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘MASS’


The following object is masked from ‘package:dplyr’:

    select




# Machine Learning in R

As stated earlier, R is a domain specific programming language for advanced analytics. Many data related tasks that are relatively hard to do in other programming languages are easier to do in R. Let's illustrate with a simple multivariate linear regression example.

We will use a data set obtained from the ***MASS*** package that contains information about home values in suburban Boston in 1978. The data set contains the following 14 variables:

- CRIM: per capita crime rate by town
- ZN: proportion of residential land zoned for lots over 25,000 sq.ft.
- INDUS: proportion of non-retail business acres per town.
- CHAS: Charles River dummy variable (1 if tract bounds river; 0 otherwise)
- NOX: nitric oxides concentration (parts per 10 million)
- RM: average number of rooms per dwelling
- AGE: proportion of owner-occupied units built prior to 1940
- DIS: weighted distances to five Boston employment centres
- RAD: index of accessibility to radial highways
- TAX: full-value property-tax rate per 10K dollars
- PTRATIO: pupil-teacher ratio by town
- B: 1000*(Bk - 0.63)^2 where Bk is the proportion of blacks by town
- LSTAT: % lower status of the population
- MEDV: Median value of owner-occupied homes in $1000's

We will use the above data set to develop a model that predicts ***MEDV*** based on ***crim***, ***rm***, ***tax***, and ***lstat***. Before we develop the model  we will use the following code to partition the data set into a training data set and testing data set using the flowing code:


In [2]:
data("Boston")

set.seed(1)
train_rownumbers <- sample(nrow(Boston), size = trunc(0.8*nrow(Boston)))

train_dataset <- 
  Boston %>%
  filter(row_number() %in% train_rownumbers) %>%
  dplyr::select(medv, crim, rm, tax, lstat)

test_dataset <- 
  Boston %>% 
  filter(!row_number() %in% train_rownumbers) %>%
  dplyr::select(medv, crim,rm,tax,lstat)  

The above code creates two data frames based on a 80/20 split. The first data frame that is created is the ***train_set*** data frame which represents 80% of the data and it will be used to train the model. The second data fame that is created is the ***test_dataset*** data frame which represents 20% of the data set and it will be used to test the model we created to see how well the model generalizes to new data. This is the simplest method to use to prevent your model from overfitting your available data as well as to test to see how well your data might generalize to new data.

Next, let's create the model using the ***train_set*** data frame:

In [3]:
model <- lm(medv ~ crim+rm+tax+lstat, data = train_dataset)
summary(model)


Call:
lm(formula = medv ~ crim + rm + tax + lstat, data = train_dataset)

Residuals:
    Min      1Q  Median      3Q     Max 
-11.035  -3.641  -1.094   1.684  30.205 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) -2.623252   3.718279  -0.706   0.4809    
crim        -0.051392   0.043930  -1.170   0.2428    
rm           5.464395   0.507473  10.768   <2e-16 ***
tax         -0.003947   0.002190  -1.803   0.0722 .  
lstat       -0.570423   0.056879 -10.029   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 5.614 on 399 degrees of freedom
Multiple R-squared:  0.6541,	Adjusted R-squared:  0.6507 
F-statistic: 188.7 on 4 and 399 DF,  p-value: < 2.2e-16


The ***lm()*** function is used above to create the model. The function requires a formula and the data set to train the model. The formula was very easy to construct. The formula is comprised of a ***dependent*** variable on the left, followed by a "~", followed by the independent ***variables***.

The next thing we want to do is see how well model generalize against new data. That can be easily accomplished using the ***predict()*** function which also comes from the base R package. To function requires two parameters; the variable that holds the model object and the variable that holds the data frame needed by the model. The output that is generated from the ***predict()*** function is a vector that contains the scores. The following code illustrates using the ***predict()*** function to score the data in ***test_dataset***:

In [4]:
data_to_score <- dplyr::select(test_dataset, crim, rm, tax, lstat)
pred_medv <- predict(model, data_to_score)
pred_medv[0:5]

1         2         3         4         5 
28.663161 21.906314  9.835655 19.194521 19.319004

We can easily add the results of the predictions to the ***test_dataset*** data frame using the code below:

In [5]:
final_output <- cbind(test_dataset, pred_medv)
final_output[0:5,]

,medv,crim,rm,tax,lstat,pred_medv
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,28.7,0.02985,6.430,222,5.21,28.663161
2,22.9,0.08829,6.012,311,12.43,21.906314
3,16.5,0.21124,5.631,311,29.93,9.835655
4,18.9,0.17004,6.004,311,17.10,19.194521
5,15.0,0.22489,6.377,311,20.45,19.319004


The code above uses the ***cbind()*** function to append the **pred_medv** column to the end of the ***test_dataset*** data frame. The "c" in ***cbind()*** stands for column so the purpose of the function is to do column based data bindings. 

We now need to test how well the new model generalize against the new ***test_dataset***. We will use the ***RMSE*** statistic ***rmse()*** function from the ***Metrics*** package. Here is the code needed to perform the task:

In [6]:
rmse_stat <- rmse(final_output$medv, final_output$pred_medv)
rmse_stat

[1] 4.865324

The ***rmse_stat*** is the standard deviation of the residuals. The lower the number the better. This metric is used to compare models. The model with the lowest ***RMSE*** stat is considered the best model when comparing models using this stat.

The above is an example of how to implement a very simple linear regression model using base R functionality. Base R also offer the ability to develop models that belongs to the ***generalized linear model*** family via the ***glm()*** function. There are also special packages in R such as the ***caret*** package that provides powerful and flexible tools for predictive modeling.